# Sprawdzenie nauczonego modelu

## Użyte biblioteki

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2

## Zmienne globalne

In [2]:
CHECK_PATH = 'check'

FIRST_PHOTO = os.path.join(CHECK_PATH, "first.jpg")
SECOND_PHOTO = os.path.join(CHECK_PATH, "second.jpg")

## Utworzenie folderu do sprawdzenia

In [ ]:
!mkdir $CHECK_PATH

## Definicja modelu

In [3]:
input_shape = (250, 250, 3)

def create_base_network(input_shape):
  input = Input(shape=input_shape)
  x = Conv2D(32, (5, 5), activation='relu')(input)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Flatten()(x)
  x = Dense(256, activation='relu')(x)
  return Model(inputs=input, outputs=x)

base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

def euclidean_distance(vects):
  x, y = vects
  return tf.sqrt(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True))

distance = Lambda(euclidean_distance, output_shape=(1,))([processed_a, processed_b])

model = Model([input_a, input_b], distance)

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.load_weights('model/model_1.h5')

## Przetworzenie obrazów wejściowych
Najlepiej je wziąć z folderu extracted

In [14]:
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (250, 250))
    img = img.astype('float32') / 255.0
    return img

img1_path = FIRST_PHOTO
img2_path = SECOND_PHOTO

img1 = preprocess_image(img1_path)
img2 = preprocess_image(img2_path)

img1 = np.expand_dims(img1, axis=0)
img2 = np.expand_dims(img2, axis=0)

pair = [img1, img2]

## Predykcja

In [ ]:
prediction = model.predict(pair)

def is_same_person(prediction, threshold=0.5):
    return prediction/1000 < threshold

result = is_same_person(prediction)
print(result[0][0])